In [1]:
import os
import xml.etree.ElementTree as ET
import cv2

annotations_path = r"C:\Users\TANYA\OneDrive\Desktop\archive\annotations"
images_path = r"C:\Users\TANYA\OneDrive\Desktop\archive\images"
output_dir = "face_mask_dataset"

categories = ["with_mask", "without_mask", "mask_weared_incorrect"]

# Create folders
for cat in categories:
    os.makedirs(os.path.join(output_dir, cat), exist_ok=True)


In [2]:
for xml_file in os.listdir(annotations_path):
    if not xml_file.endswith(".xml"):
        continue

    xml_path = os.path.join(annotations_path, xml_file)
    tree = ET.parse(xml_path)
    root = tree.getroot()

    img_filename = root.find("filename").text
    img_path = os.path.join(images_path, img_filename)
    image = cv2.imread(img_path)

    if image is None:
        continue

    for obj in root.findall("object"):
        label = obj.find("name").text
        if label not in categories:
            continue

        bbox = obj.find("bndbox")
        xmin = int(float(bbox.find("xmin").text))
        ymin = int(float(bbox.find("ymin").text))
        xmax = int(float(bbox.find("xmax").text))
        ymax = int(float(bbox.find("ymax").text))

        cropped = image[ymin:ymax, xmin:xmax]
        resized = cv2.resize(cropped, (128, 128))

        save_name = f"{img_filename[:-4]}_{label}.jpg"
        save_path = os.path.join(output_dir, label, save_name)
        cv2.imwrite(save_path, resized)


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2)

train_gen = datagen.flow_from_directory(output_dir,
                                        target_size=(128, 128),
                                        batch_size=32,
                                        class_mode='categorical',
                                        subset='training')

val_gen = datagen.flow_from_directory(output_dir,
                                      target_size=(128, 128),
                                      batch_size=32,
                                      class_mode='categorical',
                                      subset='validation')


Found 922 images belonging to 3 classes.
Found 229 images belonging to 3 classes.


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2)

train_gen = datagen.flow_from_directory(output_dir,
                                        target_size=(128, 128),
                                        batch_size=32,
                                        class_mode='categorical',
                                        subset='training')

val_gen = datagen.flow_from_directory(output_dir,
                                      target_size=(128, 128),
                                      batch_size=32,
                                      class_mode='categorical',
                                      subset='validation')


Found 922 images belonging to 3 classes.
Found 229 images belonging to 3 classes.


In [6]:
pip install opencv-python tensorflow


In [14]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import xml.etree.ElementTree as ET

# Paths to your folders
image_dir = r"C:\Users\TANYA\OneDrive\Desktop\archive\images"
annotation_dir = r"C:\Users\TANYA\OneDrive\Desktop\archive\annotations"

# Class label mapping
label_map = {'with_mask': 0, 'without_mask': 1, 'mask_weared_incorrect': 2}

# Prepare dataset
X = []
y = []

for xml_file in os.listdir(annotation_dir):
    if not xml_file.endswith('.xml'):
        continue
    tree = ET.parse(os.path.join(annotation_dir, xml_file))
    root = tree.getroot()
    
    filename = root.find('filename').text
    img_path = os.path.join(image_dir, filename)

    if not os.path.exists(img_path):
        continue
    
    image = cv2.imread(img_path)
    if image is None:
        continue

    for obj in root.findall('object'):
        label = obj.find('name').text
        if label not in label_map:
            continue
        bbox = obj.find('bndbox')
        x1 = int(bbox.find('xmin').text)
        y1 = int(bbox.find('ymin').text)
        x2 = int(bbox.find('xmax').text)
        y2 = int(bbox.find('ymax').text)

        face = image[y1:y2, x1:x2]
        face = cv2.resize(face, (128, 128))
        face = face / 255.0
        X.append(face)
        y.append(label_map[label])

X = np.array(X)
y = to_categorical(y, num_classes=3)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Save model
model.save("mask_detector_model.h5")
print("✅ Model saved as 'mask_detector_model.h5'")


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,392,707 (28.20 MB)

 Trainable params: 7,392,707 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 66s 588ms/step - accuracy: 0.8416 - loss: 0.4667 - val_accuracy: 0.9362 - val_loss: 0.1913
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 54s 529ms/step - accuracy: 0.9272 - loss: 0.2129 - val_accuracy: 0.9374 - val_loss: 0.1946
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 56s 553ms/step - accuracy: 0.9410 - loss: 0.2116 - val_accuracy: 0.9399 - val_loss: 0.1832
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 118s 1s/step - accuracy: 0.9442 - loss: 0.1832 - val_accuracy: 0.9411 - val_loss: 0.1735
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 57s 558ms/step - accuracy: 0.9331 - loss: 0.1910 - val_accuracy: 0.9387 - val_loss: 0.2035
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 52s 511ms/step - accuracy: 0.9439 - loss: 0.1640 - val_accuracy: 0.9399 - val_loss: 0.1684
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 53s 520ms/step - accuracy: 0.9373 - loss: 0.1919 - val_accuracy: 0.9423 - val_loss: 0.1749
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 54s 530ms/step - accuracy: 0.9488 - loss: 0.1

✅ Model saved as 'mask_detector_model.h5'


In [15]:
model = load_model("mask_detector_model.h5")


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import winsound

# Load the trained model
model = load_model(r"C:\Users\TANYA\OneDrive\Desktop\elevate labs\mask_detector_model.h5")

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = frame[y:y+w, x:x+h]

        # Resize to 128x128 as expected by the model
        face = cv2.resize(face, (128, 128))

        # Normalize and reshape
        face = face.astype('float32') / 255.0
        face = np.expand_dims(face, axis=0)  # Shape: (1, 128, 128, 3)

        # Predict mask or no mask
        prediction = model.predict(face)
        label = "Mask" if prediction[0][0] < 0.5 else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # Sound alert if no mask
        if label == "No Mask":
            winsound.Beep(1000, 200)

    # Display the frame
    cv2.imshow("Face Mask Detection", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow("Webcam", frame)
    
    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
